In [1]:
import time

import math
import random
import time

def sample(num_samples):
    num_inside = 0
    for _ in range(num_samples):
        x, y = random.uniform(-1, 1), random.uniform(-1, 1)
        if math.hypot(x, y) <= 1:
            num_inside += 1
    return num_inside

def approximate_pi(num_samples):
    start = time.time()
    num_inside = sample(num_samples)
    end = time.time()
    
    print("pi ~= {}".format((4*num_inside)/num_samples))

In [2]:
%%time

approximate_pi(10**8)

pi ~= 3.1414932
CPU times: user 1min 23s, sys: 17.4 ms, total: 1min 23s
Wall time: 1min 23s


In [3]:
%%time

from mpi4py import MPI
import numpy
import sys

comm = MPI.COMM_SELF.Spawn(sys.executable,
                           args=['cpi.py'],
                           maxprocs=2)

N = numpy.array(100, 'i')
comm.Bcast([N, MPI.INT], root=MPI.ROOT)
PI = numpy.array(0.0, 'd')
comm.Reduce(None, [PI, MPI.DOUBLE],
            op=MPI.SUM, root=MPI.ROOT)
print(PI)

comm.Disconnect()

3.141600986923124
CPU times: user 180 ms, sys: 72.6 ms, total: 253 ms
Wall time: 1.17 s


In [4]:
!mpirun -bind-to none -map-by slot python calculate_pi.py